In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\riswa\\Desktop\\AI\\Text-Summarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [4]:
@dataclass(frozen =True)
class DataTransformationConfig:
    root_dir:Path
    data_path:Path
    tokenizer_name:Path

In [5]:
from src.summarizer.constants import *
from src.summarizer.utils import *
from src.summarizer.logging import logger
from src.summarizer.exception import CustomException


In [13]:
class ConfigurationManager:
    def __init__(self,config_path = CONFIG_PATH,params_path = PARAMS_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])
    def get_data_transformation_config(self)-> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir= config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [14]:
import os
from src.summarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [15]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [16]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-08-15 21:21:02,358: INFO: __init__: yaml file: config\config.yaml loaded successfully]
[2024-08-15 21:21:02,363: INFO: __init__: yaml file: params.yaml loaded successfully]
[2024-08-15 21:21:02,366: INFO: __init__: created directory at: artifacts]
[2024-08-15 21:21:02,374: INFO: __init__: created directory at: artifacts/data_transformation]
c:\Users\riswa\anaconda3\envs\summarizer\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\riswa\anaconda3\envs\summarizer\lib\site-packages\transformers\tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transf